In [1]:
import os
try:
  from google.colab import drive
  drive.mount('/content/drive')

  os.chdir('/content/drive/MyDrive/GitHub/kornia')
except Exception:   # Not run on Google Colab
  import getpass
  if getpass.getuser() == 'vscode':       # Running in container
    os.chdir('/workspaces/kornia/')
  else:                                   # Running in virtual environment
    os.chdir(os.path.join(os.path.expanduser('~'), 'Documents/Git/kornia/'))

print(os.getcwd())
import sys
sys.path.insert(0, os.getcwd())

from kornia.nerf.nerf_model import MLP, NerfModel

import torch
from torchsummary import summary

import matplotlib.pyplot as plt

/home/yhollander/Documents/Git/kornia


In [2]:
n_freqs = 4
d_input = 3
mlp = MLP(d_input * (2 * n_freqs + 1))

print(summary(mlp, (10, d_input * (2 * n_freqs + 1))))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1              [-1, 10, 128]           3,584
              ReLU-2              [-1, 10, 128]               0
            Linear-3              [-1, 10, 128]          16,512
              ReLU-4              [-1, 10, 128]               0
            Linear-5              [-1, 10, 128]          16,512
              ReLU-6              [-1, 10, 128]               0
            Linear-7              [-1, 10, 128]          16,512
              ReLU-8              [-1, 10, 128]               0
            Linear-9              [-1, 10, 128]          19,968
             ReLU-10              [-1, 10, 128]               0
           Linear-11              [-1, 10, 128]          16,512
             ReLU-12              [-1, 10, 128]               0
           Linear-13              [-1, 10, 128]          16,512
             ReLU-14              [-1, 

In [3]:
print(mlp)

MLP(
  (_mlp): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=27, out_features=128, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
    )
    (3): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
    )
    (4): Sequential(
      (0): Linear(in_features=155, out_features=128, bias=True)
      (1): ReLU()
    )
    (5): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
    )
    (6): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
    )
    (7): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
    )
  )
)


In [4]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/nerf')

In [5]:
# x = torch.rand(15, 11, 27)
# writer.add_graph(mlp, x)

In [6]:
nerf_model = NerfModel(11)

In [7]:
points_3d = torch.rand(15, 11, 3)
directions = torch.rand(15, 3)

nerf_model._mlp.eval()
nerf_model.eval()

writer.add_graph(nerf_model, (points_3d, directions))

In [8]:
import torch
import torch.nn as nn
from torchsummary import summary

class Multi_input(nn.Module):
    def __init__(self):
        super(Multi_input, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
        )

    def forward(self, y, z):
        a1 = self.features(y)
        a2 = self.features(z)
        return a1, a2
    
multi_inputdevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Multi_input().to(multi_inputdevice)

summary(model, [(1, 18, 18), (1, 30, 30)])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 1, 18, 18]              10
              ReLU-2            [-1, 1, 18, 18]               0
            Conv2d-3            [-1, 1, 30, 30]              10
              ReLU-4            [-1, 1, 30, 30]               0
Total params: 20
Trainable params: 20
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 1.11
Forward/backward pass size (MB): 0.02
Params size (MB): 0.00
Estimated Total Size (MB): 1.13
----------------------------------------------------------------
